In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.stats as vale
import seaborn as sns
from sklearn import linear_model
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import statsmodels as stats



In [2]:
calendario= pd.read_csv("calendar.csv")
evaluacion=pd.read_csv("sales_train_evaluation.csv")
validacion=pd.read_csv("sales_train_validation.csv")
submision=pd.read_csv("sample_submission.csv")
precio_venta=pd.read_csv("sell_prices.csv")

In [3]:
#print(calendario)
#gap=pd.DatetimeIndex(calendario["date"])-pd.DatetimeIndex(calendario["date"].shift(1))
#print(gap.max())
#print(evaluacion)
print(validacion)
#print(precio_venta)

                                  id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_validation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_validation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_validation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_validation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_validation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id  d_1  d_2  d_3  d_4  ...  d_1904  d_1905  d_1906  \
0         CA_1       CA

In [4]:
ventas_transacc=pd.melt(validacion,id_vars=["id","item_id","dept_id","cat_id","store_id","state_id"], var_name="d",value_name="sales")
print(ventas_transacc)

                                     id        item_id    dept_id   cat_id  \
0         HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1         HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2         HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3         HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4         HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                                 ...            ...        ...      ...   
58327365    FOODS_3_823_WI_3_validation    FOODS_3_823    FOODS_3    FOODS   
58327366    FOODS_3_824_WI_3_validation    FOODS_3_824    FOODS_3    FOODS   
58327367    FOODS_3_825_WI_3_validation    FOODS_3_825    FOODS_3    FOODS   
58327368    FOODS_3_826_WI_3_validation    FOODS_3_826    FOODS_3    FOODS   
58327369    FOODS_3_827_WI_3_validation    FOODS_3_827    FOODS_3    FOODS   

         store_id state_id       d  sales  
0            CA_1  

In [9]:
cat_articulos=validacion[["item_id","dept_id","cat_id"]].drop_duplicates()
cat_articulos["num_item_id"]=[i+1 for i in range(cat_articulos.shape[0])]
print(cat_articulos)

cat_tiendas=validacion[["store_id","state_id"]].drop_duplicates()
cat_tiendas["num_store"]=[i+1 for i in range(cat_tiendas.shape[0])]
print(cat_tiendas)


            item_id    dept_id   cat_id  num_item_id
0     HOBBIES_1_001  HOBBIES_1  HOBBIES            1
1     HOBBIES_1_002  HOBBIES_1  HOBBIES            2
2     HOBBIES_1_003  HOBBIES_1  HOBBIES            3
3     HOBBIES_1_004  HOBBIES_1  HOBBIES            4
4     HOBBIES_1_005  HOBBIES_1  HOBBIES            5
...             ...        ...      ...          ...
3044    FOODS_3_823    FOODS_3    FOODS         3045
3045    FOODS_3_824    FOODS_3    FOODS         3046
3046    FOODS_3_825    FOODS_3    FOODS         3047
3047    FOODS_3_826    FOODS_3    FOODS         3048
3048    FOODS_3_827    FOODS_3    FOODS         3049

[3049 rows x 4 columns]


In [6]:
validacion_num= validacion.merge(cat_articulos[["num_item_id","item_id"]],on="item_id")
validacion_num= validacion_num.merge(cat_tiendas[["store_id","num_store"]],on="store_id")
validacion_num=validacion_num.drop(["id","item_id","dept_id","cat_id","store_id","state_id"],axis=1)
venta_trans_num= pd.melt(validacion_num,id_vars=["num_item_id","num_store"] , var_name= "d",value_name="sales")

print(venta_trans_num)

          num_item_id  num_store       d  sales
0                   1          1     d_1      0
1                   2          1     d_1      0
2                   3          1     d_1      0
3                   4          1     d_1      0
4                   5          1     d_1      0
...               ...        ...     ...    ...
58327365         3045         10  d_1913      1
58327366         3046         10  d_1913      0
58327367         3047         10  d_1913      0
58327368         3048         10  d_1913      3
58327369         3049         10  d_1913      0

[58327370 rows x 4 columns]


In [7]:
print(ventas_transacc.info())
print(venta_trans_num.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 8 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   id        object
 1   item_id   object
 2   dept_id   object
 3   cat_id    object
 4   store_id  object
 5   state_id  object
 6   d         object
 7   sales     int64 
dtypes: int64(1), object(7)
memory usage: 3.5+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   num_item_id  int64 
 1   num_store    int64 
 2   d            object
 3   sales        int64 
dtypes: int64(3), object(1)
memory usage: 1.7+ GB
None


In [16]:
venta_trans_num.to_csv("Datos_limpios/ventas_transacc.csv")
cat_tiendas.to_csv("Datos_limpios/cat_tiendas.csv")
cat_articulos.to_csv("Datos_limpios/cat_articulos.csv")